In [3]:
import pandas as pd

# --- Load dataset ---
df = pd.read_csv("raw_data/daily_sales_no_log.csv")

# Ensure date is datetime
df['date'] = pd.to_datetime(df['date'], format="%d/%m/%Y")

# Sort correctly
df = df.sort_values(['date', 'time_bucket']).reset_index(drop=True)

# Identify pizza target columns (everything except the first 3 non-pizza cols)
non_pizza_cols = ['date', 'time_bucket', 'oil_price', 'is_holiday']
pizza_cols = [c for c in df.columns if c not in non_pizza_cols]

# --- Step 1: Pivot to get daily buckets for lag/rolling ---
pivot = df.pivot(index='date', columns='time_bucket', values=pizza_cols)

# Flatten multi-index column
pivot.columns = [f"{col}_{tb}" for col, tb in pivot.columns]

# --- Step 2: Yesterday (lag1) ---
lag1 = pivot.shift(1)
lag1.columns = [f"{c}_yesterday" for c in lag1.columns]

# --- Step 3: 3-day rolling averages ---
roll3 = pivot.rolling(3).mean()
roll3.columns = [f"{c}_avg3" for c in roll3.columns]

# --- Step 4: Merge ONLY lag + rolling features back to original df ---
df_aug = df.merge(lag1, on='date', how='left')
df_aug = df_aug.merge(roll3, on='date', how='left')

# --- Step 5: Ensure pizza targets remain on the far right ---
final_cols = [c for c in df_aug.columns if c not in pizza_cols] + pizza_cols
df_aug = df_aug[final_cols]

# Impute any NaNs created by lag/rolling with mean of the column
for col in df_aug.columns:
    if df_aug[col].isna().sum() > 0:
        mean_value = df_aug[col].mean()
        df_aug[col].fillna(mean_value, inplace=True)

# Save
df_aug.to_csv("modelling data/daily_sales.csv", index=False)

df_aug.head()


/var/folders/xl/t0lmyr3n2zj7_lvsrc1d1bv40000gn/T/ipykernel_12410/489749743.py:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_aug[col].fillna(mean_value, inplace=True)


,date,time_bucket,oil_price,is_holiday,bbq_ckn_l_Afternoon_yesterday,bbq_ckn_l_Dinner_yesterday,bbq_ckn_l_Lunch_yesterday,bbq_ckn_m_Afternoon_yesterday,bbq_ckn_m_Dinner_yesterday,bbq_ckn_m_Lunch_yesterday,...,thai_ckn_m,thai_ckn_s,the_greek_l,the_greek_m,the_greek_s,the_greek_xl,the_greek_xxl,veggie_veg_l,veggie_veg_m,veggie_veg_s
0,2015-01-01,Afternoon,52.32,1,0.787115,0.921569,1.056022,0.731092,0.957983,0.977591,...,1,0,0,0,1,0,0,0,0,0
1,2015-01-01,Dinner,52.32,1,0.787115,0.921569,1.056022,0.731092,0.957983,0.977591,...,1,0,0,0,0,1,0,0,0,1
2,2015-01-01,Lunch,52.32,1,0.787115,0.921569,1.056022,0.731092,0.957983,0.977591,...,0,0,0,0,3,0,0,1,0,1
3,2015-01-02,Afternoon,55.38,0,2.000000,2.000000,2.000000,1.000000,2.000000,1.000000,...,0,1,0,2,0,0,0,0,0,0
4,2015-01-02,Dinner,55.38,0,2.000000,2.000000,2.000000,1.000000,2.000000,1.000000,...,1,0,0,1,2,1,0,4,0,0


In [2]:
import pandas as pd

df = pd.read_csv("modelling data/daily_sales.csv")

pizza = pd.read_csv("modelling data/pizzas.csv")

# For each pizza_id in pizza that is not in df, add a column of all 0 to df
for p in pizza['pizza_id']:
    if p not in df.columns:
        df[p] = 0

# Save
df.to_csv("modelling data/daily_sales.csv", index=False)
